In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, Run

print(azureml.core.VERSION)

1.0.17


In [1]:
from azureml.core import Workspace,Experiment ,Run
ws = Workspace.get(name='###',
                      subscription_id='###', 
                      resource_group='###' 
                     )

In [2]:
ws.get_details()

In [4]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

X, y = load_diabetes(return_X_y = True)
columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data = {
    "train":{"X": X_train, "y": y_train},        
    "test":{"X": X_test, "y": y_test}
}

print ("Data contains", len(data['train']['X']), "training samples and",len(data['test']['X']), "test samples")

Data contains 353 training samples and 89 test samples


In [5]:
# Create, fit, and test the scikit-learn Ridge regression model
regression_model = Ridge(alpha=0.03)
regression_model.fit(data['train']['X'], data['train']['y'])
preds = regression_model.predict(data['test']['X'])

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))

Mean Squared Error is 3424.900315896017


In [6]:
from sklearn.externals import joblib

joblib.dump(value=regression_model, filename='sklearn_regression_model.pkl')

['sklearn_regression_model.pkl']

In [7]:
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl",
                       model_name = "sklearn_regression_model.pkl",
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

Registering model sklearn_regression_model.pkl


In [8]:
%%writefile score.py
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any datatype if it is JSON-serializable
        return result.tolist()
        return 0
    except Exception as e:
        error = str(e)
        return error

Writing score.py


In [9]:
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [10]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "diabetes", 'type': "regression"},
                                 description = "Image with ridge regression model")

image = Image.create(name = "myimage1",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

print('Execution complete')

Creating image
Running..................................................
SucceededImage creation operation finished for image myimage1:1, operation "Succeeded"
Execution complete


In [11]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'sample name': 'AML 101'}, 
                                               description='This is a great example.')

In [12]:
%%time
from azureml.core.webservice import Webservice

# Create the webservice 
service = Webservice.deploy_from_model(name='my-aci-svc3',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config,
                                       workspace=ws)

# Wait for the service deployment to complete while displaying log output.  This can take several minutes.
service.wait_for_deployment(show_output=True)

print('Execution complete')

Creating image
Image creation operation finished for image my-aci-svc3:1, operation "Succeeded"
Creating service
Running....................
SucceededACI service creation operation finished, operation "Succeeded"
Execution complete
CPU times: user 1.79 s, sys: 134 ms, total: 1.92 s
Wall time: 6min 51s


In [13]:
import json

# scrape the first row from the test set.
test_samples = json.dumps({"data": X_test[0:1, :].tolist()})

print(test_samples) # here is what we are sending to the service.

#score on our service
service.run(input_data = test_samples)

{"data": [[0.0199132141783263, 0.0506801187398187, 0.104808689473925, 0.0700725447072635, -0.0359677812752396, -0.0266789028311707, -0.0249926566315915, -0.00259226199818282, 0.00371173823343597, 0.0403433716478807]]}


[235.1137108158503]

In [14]:
ws.delete(delete_dependent_resources=True)